In [1]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install transformers==4.21.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import wandb
import os
from datasets import DatasetDict, load_dataset, load_metric

In [6]:
USER = "siddhantm"
WANDB_PROJECT = "capstone_jpmc"
wandb.init(entity=USER, project=WANDB_PROJECT)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: siddhantm. Use `wandb login --relogin` to force relogin


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# PROJECT_ROOT = "/content/drive/MyDrive/JPMC Capstone/Speech_Emotion"
PROJECT_ROOT = "/content/drive/MyDrive/JPMC Capstone/Audio-Classification-HF-main"

In [9]:
!pwd

/content


In [10]:
os.chdir(PROJECT_ROOT)

In [11]:
!pwd

/content/drive/.shortcut-targets-by-id/1_cBqTwetZCYD3uVjVHeosGtR7NkaEAwE/JPMC Capstone/Audio-Classification-HF-main


In [12]:
import os
import logging
import librosa
# import src.utils

import numpy as np

from datasets import DatasetDict, load_dataset, load_metric
from transformers import (
    Wav2Vec2ForSequenceClassification,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    Wav2Vec2FeatureExtractor,
)
from src.utils import collator

logging.basicConfig(
    format="%(asctime)s | %(levelname)s: %(message)s", level=logging.INFO
)

In [13]:
from transformers import Wav2Vec2FeatureExtractor
model = "facebook/wav2vec2-base"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model)
print(feature_extractor)

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}



In [14]:
NUM_LABELS = 6
model_id = "facebook/wav2vec2-base"

# config = PretrainedConfig.from_pretrained(model_id, num_labels=NUM_LABELS)
# hubert_model = Wav2Vec2ForSequenceClassification.from_pretrained(
#     model_id,
#     config=config  # because we need to update num_labels as per our dataset
#     # ignore_mismatched_sizes=True,  # to avoid classifier size mismatch from from_pretrained.
# )

wav2vec_model = Wav2Vec2ForSequenceClassification.from_pretrained(model_id, num_labels=NUM_LABELS)

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForSequenceClassification: ['project_hid.weight', 'project_hid.bias', 'quantizer.weight_proj.bias', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_q.weight', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertFor

In [15]:
# freeze all layers to begin with
for param in wav2vec_model.parameters():
    param.requires_grad = False
    
# freeze two encoder layers    
layers_freeze_num = 2
n_layers = (
    4 + layers_freeze_num * 16
)  # 4 refers to projector and classifier's weights and biases.
for name, param in list(wav2vec_model.named_parameters())[-n_layers:]:
    param.requires_grad = True

In [16]:
dataset_config = {
  "LOADING_SCRIPT_FILES": os.path.join(PROJECT_ROOT, "src/data/crema.py"),
  "CONFIG_NAME": "clean",
  "DATA_DIR": os.path.join(PROJECT_ROOT, "data/archive.zip"),
  "CACHE_DIR": os.path.join(PROJECT_ROOT, "cache_crema"),
}
ds = load_dataset(
  dataset_config["LOADING_SCRIPT_FILES"],
  dataset_config["CONFIG_NAME"],
  data_dir=dataset_config["DATA_DIR"],
  cache_dir=dataset_config["CACHE_DIR"]
)
print(ds)

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'label'],
        num_rows: 7442
    })
})


In [17]:
# CONVERING RAW AUDIO TO ARRAYS
ds = ds.map(
        lambda x: {
            "array": librosa.load(x["file"], sr=16000, mono=False)[0]
        },
        num_proc=2,
    )

In [18]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['file', 'label', 'array'],
        num_rows: 7442
    })
})


In [19]:
# PROCESS THE DATASET TO THE FORMAT EXPECTED BY THE MODEL FOR TRAINING

INPUT_FIELD = "input_values"
LABEL_FIELD = "labels"

def prepare_dataset(batch, feature_extractor):
    audio_arr = batch["array"]
    input = feature_extractor(
        audio_arr, sampling_rate=16000, padding=True, return_tensors="pt"
    )

    batch[INPUT_FIELD] = input.input_values[0]
    batch[LABEL_FIELD] = batch[
        "label"
    ]  # colname MUST be labels as Trainer will look for it by default

    return batch

In [ ]:
ds["train"][0]

In [22]:
# APPLY THE DATA PREP USING FEATURE EXTRACTOR TO ALL EXAMPLES
ds = ds.map(
    prepare_dataset,
    fn_kwargs={"feature_extractor": feature_extractor},
    # num_proc=2,
)
logging.info("Finished extracting features from audio arrays.")

In [23]:
# LABEL TO ID
ds = ds.class_encode_column("labels")

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [24]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['file', 'label', 'array', 'input_values', 'labels'],
        num_rows: 7442
    })
})


In [25]:
# INTRODUCE TRAIN TEST VAL SPLITS

# 90% train, 10% test + validation
train_testvalid = ds["train"].train_test_split(shuffle=True, test_size=0.1)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
ds = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'val': test_valid['train']})

In [26]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['file', 'label', 'array', 'input_values', 'labels'],
        num_rows: 6697
    })
    test: Dataset({
        features: ['file', 'label', 'array', 'input_values', 'labels'],
        num_rows: 373
    })
    val: Dataset({
        features: ['file', 'label', 'array', 'input_values', 'labels'],
        num_rows: 372
    })
})


In [27]:
trainer_config = {
  "OUTPUT_DIR": "results",
  "TRAIN_EPOCHS": 5,
  "TRAIN_BATCH_SIZE": 8,
  "EVAL_BATCH_SIZE": 8,
  "GRADIENT_ACCUMULATION_STEPS": 4,
  "WARMUP_STEPS": 500,
  "DECAY": 0.01,
  "LOGGING_STEPS": 10,
  "MODEL_DIR": "models/test-hubert-model",
  "SAVE_STEPS": 100
}

# Fine-Tuning with Trainer
training_args = TrainingArguments(
    output_dir=trainer_config["OUTPUT_DIR"],  # output directory
    gradient_accumulation_steps=trainer_config[
        "GRADIENT_ACCUMULATION_STEPS"
    ],  # accumulate the gradients before running optimization step
    num_train_epochs=trainer_config[
        "TRAIN_EPOCHS"
    ],  # total number of training epochs
    per_device_train_batch_size=trainer_config[
        "TRAIN_BATCH_SIZE"
    ],  # batch size per device during training
    per_device_eval_batch_size=trainer_config[
        "EVAL_BATCH_SIZE"
    ],  # batch size for evaluation
    warmup_steps=trainer_config[
        "WARMUP_STEPS"
    ],  # number of warmup steps for learning rate scheduler
    save_steps=trainer_config["SAVE_STEPS"], # save checkpoint every 100 steps
    weight_decay=trainer_config["DECAY"],  # strength of weight decay
    logging_steps=trainer_config["LOGGING_STEPS"],
    evaluation_strategy="epoch",  # report metric at end of each epoch
    report_to="wandb",  # enable logging to W&B
)

In [28]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union

import torch
from transformers import Wav2Vec2Processor

INPUT_FIELD = "input_values"
LABEL_FIELD = "labels"


@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(
        self, examples: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:

        input_features = [
            {INPUT_FIELD: example[INPUT_FIELD]} for example in examples
        ]  # example is basically row0, row1, etc...
        labels = [example[LABEL_FIELD] for example in examples]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        batch[LABEL_FIELD] = torch.tensor(labels)

        return batch

In [29]:
# DEFINE DATA COLLATOR - TO PAD TRAINING BATCHES DYNAMICALLY
data_collator = DataCollatorCTCWithPadding(
            processor=feature_extractor,
            padding=True
)

In [30]:
from datasets import load_metric

def compute_metrics(eval_pred):
    # DEFINE EVALUATION METRIC
    compute_accuracy_metric = load_metric("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return compute_accuracy_metric.compute(predictions=predictions, references=labels)

In [31]:
# START TRAINING
trainer = Trainer(
    model=wav2vec_model,  # the instantiated 🤗 Transformers model to be trained
    args=training_args,  # training arguments, defined above
    data_collator=data_collator,
    train_dataset=ds["train"],  # training dataset
    eval_dataset=ds["val"],  # evaluation dataset
    compute_metrics=compute_metrics,
)

In [32]:
trainer.train()

# TO RESUME TRAINING FROM CHECKPOINT
# trainer.train("results/checkpoint-2000")

The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForSequenceClassification.forward` and have been ignored: file, array. If file, array are not expected by `Wav2Vec2ForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6697
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 1045
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy
0,1.461300,1.499951,0.370968
1,1.237900,1.344728,0.473118
2,1.091900,1.204831,0.551075
3,0.818500,1.195790,0.559140
4,0.847000,1.073772,0.599462


Saving model checkpoint to results/checkpoint-100
Configuration saved in results/checkpoint-100/config.json
Model weights saved in results/checkpoint-100/pytorch_model.bin
Saving model checkpoint to results/checkpoint-200
Configuration saved in results/checkpoint-200/config.json
Model weights saved in results/checkpoint-200/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForSequenceClassification.forward` and have been ignored: file, array. If file, array are not expected by `Wav2Vec2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 372
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """


Saving model checkpoint to results/checkpoint-300
Configuration saved in results/checkpoint-300/config.json
Model weights saved in results/checkpoint-300/pytorch_model.bin
Saving model checkpoint to results/checkpoint-400
Configuration saved in results/checkpoint-400/config.json
Model weights saved in results/checkpoint-400/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForSequenceClassification.forward` and have been ignored: file, array. If file, array are not expected by `Wav2Vec2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 372
  Batch size = 8
Saving model checkpoint to results/checkpoint-500
Configuration saved in results/checkpoint-500/config.json
Model weights saved in results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to results/checkpoint-600
Configuration saved in results/checkpoint-600/config.json
Model weights saved in resu

TrainOutput(global_step=1045, training_loss=1.1927522312511096, metrics={'train_runtime': 2503.2826, 'train_samples_per_second': 13.376, 'train_steps_per_second': 0.417, 'total_flos': 1.0126927891272957e+18, 'train_loss': 1.1927522312511096, 'epoch': 5.0})

In [33]:
  # TEST RESULTS
test_results = trainer.predict(ds["test"])
logging.info("Test Set Result: {}".format(test_results.metrics))
wandb.log({"test_accuracy": test_results.metrics["test_accuracy"]})

trainer.save_model(os.path.join(PROJECT_ROOT, trainer_config["MODEL_DIR"]))

The following columns in the test set don't have a corresponding argument in `Wav2Vec2ForSequenceClassification.forward` and have been ignored: file, array. If file, array are not expected by `Wav2Vec2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 373
  Batch size = 8


Saving model checkpoint to /content/drive/MyDrive/JPMC Capstone/Audio-Classification-HF-main/models/test-hubert-model
Configuration saved in /content/drive/MyDrive/JPMC Capstone/Audio-Classification-HF-main/models/test-hubert-model/config.json
Model weights saved in /content/drive/MyDrive/JPMC Capstone/Audio-Classification-HF-main/models/test-hubert-model/pytorch_model.bin


In [34]:
test_results

PredictionOutput(predictions=array([[-1.7910028 ,  2.555515  , -0.02263894, -1.5943929 , -0.8777184 ,
         2.0647745 ],
       [-1.455168  , -0.42449498, -1.235836  , -0.2666624 ,  3.3182604 ,
         0.8253097 ],
       [-2.04499   ,  1.3799231 , -0.67204225, -1.5797179 ,  1.4355501 ,
         2.1995783 ],
       ...,
       [-1.4638634 , -2.186478  ,  1.8454161 ,  2.1463037 , -0.31156504,
        -0.6929132 ],
       [-0.21178316,  3.145608  , -1.1225017 , -0.12890658, -1.1253959 ,
        -0.49052155],
       [-1.5236598 , -1.8439506 ,  2.414146  ,  1.2641914 , -0.5923106 ,
        -0.2673627 ]], dtype=float32), label_ids=array([5, 1, 5, 5, 3, 0, 3, 0, 2, 3, 4, 5, 3, 1, 3, 4, 2, 3, 5, 5, 0, 5,
       3, 4, 5, 0, 4, 4, 1, 2, 1, 3, 5, 1, 0, 1, 3, 0, 1, 3, 1, 3, 3, 4,
       3, 1, 4, 0, 4, 2, 0, 3, 2, 1, 1, 0, 5, 1, 5, 0, 1, 4, 0, 3, 1, 0,
       1, 3, 5, 5, 1, 2, 3, 2, 2, 4, 3, 1, 3, 3, 5, 1, 2, 2, 0, 4, 3, 2,
       0, 2, 5, 3, 3, 4, 3, 3, 3, 5, 4, 0, 4, 4, 1, 2, 5, 5, 3, 3, 2, 

In [ ]:
# logging trained models to wandb
wandb.save(
    os.path.join(PROJECT_ROOT, trainer_config["MODEL_DIR"], "*"),
    base_path=os.path.dirname(trainer_config["MODEL_DIR"]),
    policy="end",
)

ValueError: ignored